# HW3 
# Hannah Wang

## Question 1

### (a)


$dC_t = \frac{\partial C}{\partial r_t}dr_t+\frac{\partial C}{\partial t}dt+\frac{1}{2}\frac{\partial^2 C}{\partial r_t^2}(dr_t)^2$

$= \alpha(r_t,t)\frac{\partial C}{\partial r_t}dt+\beta(r_t,t)\frac{\partial C}{\partial r_t}dW_t+\frac{\partial C}{\partial t}dt+\frac{1}{2}\frac{\partial^2 C}{\partial r_t^2}\beta^2(r_t,t)dt$

$=(\alpha(r_t,t)\frac{\partial C}{\partial r_t}+\frac{1}{2}\frac{\partial^2 C}{\partial r_t^2}\beta^2(r_t,t)+\frac{\partial C}{\partial t})dt+\beta(r_t,t)\frac{\partial C}{\partial r_t}dW_t$

Setting drift equal to $rC$, we get:

$$
\alpha(r_t,t)\frac{\partial C}{\partial r_t}+\frac{1}{2}\frac{\partial^2 C}{\partial r_t^2}\beta^2(r_t,t)+\frac{\partial C}{\partial t}=rC
$$

### (b)

$\kappa(\theta-r_t)\frac{\partial C}{\partial r_t}+\frac{1}{2}\frac{\partial^2 C}{\partial r_t^2}\sigma^2+\frac{\partial C}{\partial t}=rC$

$q_u = \frac{1}{2}\big[\frac{\kappa(\theta-r_t)\Delta t}{\Delta r_t}+\frac{\sigma^2\Delta t}{\Delta r_t^2}\big]$

$q_m = 1-\frac{\sigma^2\Delta t}{(\Delta r_t)^2}$

$q_d = \frac{1}{2}\big[-\frac{\kappa(\theta-r_t)\Delta t}{\Delta r_t}+\frac{\sigma^2\Delta t}{(\Delta r_t)^2}\big]$

In [18]:
import numpy as np

In [19]:
class Vasicek:
    
    def __init__(self,kappa,theta,sigma): 
        self.kappa=kappa
        self.theta=theta
        self.sigma=sigma

In [20]:
hw3dynamics = Vasicek(kappa=3,theta=0.05,sigma=0.03)

In [21]:
class Bond:
    
    def __init__(self, T):
        self.T=T
    

In [22]:
hw3contract = Bond(T=5)

In [23]:
class FDexplicit:
    
    def __init__(self, rMax, rMin, deltar, deltat, useUpwind):         
        self.rMax=rMax
        self.rMin=rMin
        self.deltar=deltar
        self.deltat=deltat
        self.useUpwind=useUpwind
        
    
    def price_bond_vasicek(self,contract,dynamics):
    # You complete the coding of this function
    #
    # Returns array of all initial short rates,
    # and the corresponding array of zero-coupon
    # T-maturity bond prices

        T = contract.T
        N=round(T/self.deltat)
        if abs(N-T/self.deltat) > 1e-12:
            raise ValueError("Bad delta t")

        r=np.arange(self.rMax,self.rMin-self.deltar/2,-self.deltar)   #I'm making the FIRST indices of the array correspond to HIGH levels of r
        bondprice=np.ones(np.size(r))
        a=(dynamics.kappa*(dynamics.theta-r)*self.deltat/self.deltar)
        b=(dynamics.sigma**2*self.deltat/(self.deltar**2))

        if self.useUpwind:
            qu=    0.5*(b+a+abs(a))#fill this in with an array.
            qd=    0.5*(b-a+abs(a)) #fill this in with an array.
            qm=    1-qu-qd #fill this in with an array.
        else:
            qu=    0.5*(a+b)#fill this in with an array.
            qd=    0.5*(b-a)#fill this in with an array.
            qm=    1-b#fill this in with an array.

        for t in np.arange(N-1,-1,-1)*self.deltat:
            bondprice=1/(1+r*self.deltat)*(qd*np.roll(bondprice,-1)+qm*bondprice+qu*np.roll(bondprice,1))

            # It is not obvious in this case, 
            # what boundary conditions to use at the top and bottom
            # so let us assume "linearity" boundary conditions
            bondprice[0]=2*bondprice[1]-bondprice[2]
            bondprice[-1]=2*bondprice[-2]-bondprice[-3]

        return (r, bondprice)

In [24]:
hw3FD1 = FDexplicit(rMax=0.35,rMin=-0.25,deltar=0.01,deltat=0.01,useUpwind=False)

In [25]:
(r1, bondprice1) = hw3FD1.price_bond_vasicek(hw3contract,hw3dynamics)

In [26]:
np.set_printoptions(precision=4,suppress=True)
displayrows1=(r1<0.15+hw3FD1.deltar/2) & (r1>0.0-hw3FD1.deltar/2)

In [28]:
print(np.stack((r1, bondprice1),1)[displayrows1])

[[ 1.5000e-01 -1.4273e+09]
 [ 1.4000e-01  1.6361e+08]
 [ 1.3000e-01  2.2294e+07]
 [ 1.2000e-01 -1.3724e+06]
 [ 1.1000e-01 -1.3361e+05]
 [ 1.0000e-01  3.2966e+03]
 [ 9.0000e-02  1.3021e+02]
 [ 8.0000e-02  7.7128e-01]
 [ 7.0000e-02  7.7385e-01]
 [ 6.0000e-02  7.7643e-01]
 [ 5.0000e-02  7.7902e-01]
 [ 4.0000e-02  7.8162e-01]
 [ 3.0000e-02  7.8423e-01]
 [ 2.0000e-02  7.8685e-01]
 [ 1.0000e-02  1.4165e+03]
 [-3.3307e-16  5.1498e+04]]


### (c) 

When $\kappa(\theta-r_j)\ge 0$

$\kappa(\theta-r_j)\frac{C^{j+1}_{n+1}-C^{j}_{n+1}}{\Delta r}+\frac{1}{2}\sigma^2\frac{C^{j+1}_{n+1}-2C^{j}_{n+1}+C^{j-1}_{n+1}}{(\Delta r)^2}+\frac{C^{j}_{n+1}-C^{j}_{n}}{\Delta t}=rC_{n}^{j}$

$q_u = \frac{\kappa(\theta-r_j)\Delta t}{\Delta r}+\frac{\sigma^2\Delta t}{2(\Delta r)^2}$

$q_m = 1-\frac{\sigma^2\Delta t}{(\Delta r)^2}-\frac{\kappa(\theta-r_j)\Delta t}{\Delta r}$

$q_d = \frac{\sigma^2\Delta t}{2(\Delta r)^2}$

When $\kappa(\theta-r_j) < 0$

$\kappa(\theta-r_j)\frac{C^{j}_{n+1}-C^{j-1}_{n+1}}{\Delta r}+\frac{1}{2}\sigma^2\frac{C^{j+1}_{n+1}-2C^{j}_{n+1}+C^{j-1}_{n+1}}{(\Delta r)^2}+\frac{C^{j}_{n+1}-C^{j}_{n}}{\Delta t}=rC_{n}^{j}$

$q_u = \frac{\sigma^2\Delta t}{2(\Delta r)^2}$

$q_m = 1-\frac{\sigma^2\Delta t}{(\Delta r)^2}+\frac{\kappa(\theta-r_j)\Delta t}{\Delta r}$

$q_d = \frac{\sigma^2\Delta t}{2(\Delta r)^2}-\frac{\kappa(\theta-r_j)\Delta t}{\Delta r}$

In [30]:
hw3FD2 = FDexplicit(rMax=0.35,rMin=-0.25,deltar=0.01,deltat=0.01,useUpwind=True)

In [31]:
(r2, bondprice2) = hw3FD2.price_bond_vasicek(hw3contract,hw3dynamics)

In [32]:
np.set_printoptions(precision=4,suppress=True)
displayrows2=(r2<0.15+hw3FD2.deltar/2) & (r2>0.0-hw3FD2.deltar/2)

In [33]:
print(np.stack((r2, bondprice2),1)[displayrows2])

[[ 0.15    0.7536]
 [ 0.14    0.7561]
 [ 0.13    0.7586]
 [ 0.12    0.7611]
 [ 0.11    0.7637]
 [ 0.1     0.7662]
 [ 0.09    0.7688]
 [ 0.08    0.7713]
 [ 0.07    0.7739]
 [ 0.06    0.7765]
 [ 0.05    0.7791]
 [ 0.04    0.7817]
 [ 0.03    0.7843]
 [ 0.02    0.7869]
 [ 0.01    0.7895]
 [-0.      0.7922]]


### (d) 

By Taylor's theorem:

$f(x+h)=f(x)+hf'(x)+\frac{1}{2}h^2f''(x)+O(h^3)$

$f(x-h)=f(x)-hf'(x)+\frac{1}{2}h^2f''(x)+O(h^3)$

$\big|\frac{f(x+h)-f(x)}{h}-f'(x)\big|=\frac{1}{2}hf''(x)+\frac{O(h^3)}{h}=O(h)$

$\big|\frac{f(x+h)-f(x-h)}{2h}-f'(x)\big|=\frac{O(h^3)}{h}=O(h^2)$

### (e) 

In [34]:
(np.stack((r1, bondprice1),1)[displayrows1])[abs((np.stack((r1, bondprice1),1)[displayrows1])[:,0]-0.1)<1e-12][0,1]

3296.5929237489718

In [35]:
(np.stack((r2, bondprice2),1)[displayrows2])[abs((np.stack((r2, bondprice2),1)[displayrows2])[:,0]-0.1)<1e-12][0,1]

0.7662252882450523

The upwind calculation is more accurate.

### (f) 

less;greater

### (g) 

In [61]:
#0.12 case
ytm_1 = np.log(abs(1/((np.stack((r2, bondprice2),1)[displayrows2])[abs((np.stack((r1, bondprice1),1)[displayrows1])[:,0]-0.12)<1e-12][0,1])))/5.0
ytm_1

0.05458660238621129

In [62]:
#0.02 case
ytm_2 = np.log(abs(1/((np.stack((r2, bondprice2),1)[displayrows2])[abs((np.stack((r1, bondprice1),1)[displayrows1])[:,0]-0.02)<1e-12][0,1])))/5.0
ytm_2

0.04792684524038049

Since $\theta = 0.05$, then for $r_0 = 0.12 > 0.05 = \theta$, we can get $r_0>ytm>\theta$. For $r_0 = 0.02$, $r_0 < ytm < \theta$.

## Question 2

### (a) 

We know that

$\sigma_{imp}=\bar{\sigma_t}=\sqrt{\frac{1}{T}\int_0^T\sigma^2(t)dt}$
It is related to $T$ but not $K$.

Thus, the dynamics can generate non-constant term-structure of implied volatility but not an implied volatility skew.

### (b) 

In [41]:
from scipy.stats import norm
from scipy.optimize import bisect, brentq

In [46]:
def BSprice(sigma,S,rGrow,r,K,T):
        
    F = S*np.exp(rGrow*T)
    sd = sigma*np.sqrt(T)
    d1 = np.log(F/K)/sd+sd/2
    d2 = d1-sd
    return np.exp(-r*T)*(F*norm.cdf(d1)-K*norm.cdf(d2))

In [53]:
def IV(C,S,rGrow,r,K,T):
    df = np.exp(-r*T)
    F = S / df
    lowerbound = np.max([0,(F-K)*df])
    if C<lowerbound:
        return np.nan
    if C==lowerbound:
        return 0
    if C>=F*np.exp(-r*T):
        return np.nan 
    
    sigma = 0.2
    while BSprice(sigma,S,rGrow,r,K,T)>C:
        sigma /= 2
    while BSprice(sigma,S,rGrow,r,K,T)<C:
        sigma *= 2
    hi = sigma
    lo = hi/2
    function = lambda x: BSprice(x,S,rGrow,r,K,T)-C
    impliedVolatility = brentq(function , a=lo, b=hi, xtol=1e-4)
    return impliedVolatility
    


In [54]:
S = 100
K = 100
rGrow = 0.05
r = 0.05
T = [0.1,0.2,0.5]
C = [5.25,7.25,9.5]
imp_vols = []
for i in range(3):
    imp_vols.append(IV(C[i],S,rGrow,r,K,T[i]))
print(imp_vols)

[0.39732050375365563, 0.38017110912025387, 0.29509733829743456]



$\bar{\sigma_t}=\sqrt{\frac{1}{T}\int_0^T\sigma^2(t)dt}$


$\sigma(t)=\begin{cases} 
      \bar\sigma_1 & 0 \leq t\leq 0.1 \\
      \sqrt{2\bar\sigma_2^2-\bar\sigma_1^2} & 0.1 < t \leq 0.2\\
      \sqrt{\frac{1}{3}(5\bar\sigma_3^2-2\bar\sigma_2^2)} & 0.2 < t \leq 0.5
\end{cases}$


### (c) 

$\bar\sigma_{0.4}=\sqrt{\frac{1}{0.4}\int_0^{0.4}\sigma^2(t)dt}$

$=\sqrt{\frac{1}{0.4}*(0.1\bar\sigma_1^2+0.1(2\bar\sigma_2^2-\bar\sigma_1^2)+0.2(\frac{1}{3}(5\bar\sigma_3^2-2\bar\sigma_2^2))}$

$=\sqrt{\frac{5}{6}\bar\sigma_3^2+\frac{1}{6}\bar\sigma^2_2}$


In [63]:
vol_c = (((5/6)*imp_vols[2]**2)+((1/6)*imp_vols[1]**2))** 0.5
T_c = 0.4
price = BSprice(vol_c,S,rGrow,r,K,T_c)
print("The time-0 price of an at-the-money European call with expiry 0.4 is %s" %price)

The time-0 price of an at-the-money European call with expiry 0.4 is 8.78420254114681
